In [1]:
import pypsa
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
n=pypsa.Network()

#Snapshots
n.set_snapshots(value=pd.date_range(freq="h", start="2013-01-01", end="2014-01-01", inclusive='left'))
n.snapshot_weightings[:] *= 8760.0 / n.snapshot_weightings.sum()
Nyears = n.snapshot_weightings.objective.sum() / 8760.0

# # Load yaml files
import yaml

with open(r'config.yaml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
 
        print(config)

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages

{'version': '0.0.2', 'tutorial': True, 'logging': {'level': 'INFO', 'format': '%(levelname)s:%(name)s:%(message)s'}, 'scenario': {'simpl': [''], 'll': ['copt'], 'clusters': [6], 'opts': ['Co2L-4H']}, 'countries': ['NG'], 'summary_dir': 'results', 'snapshots': {'start': '2013-03-1', 'end': '2013-03-7', 'inclusive': 'left'}, 'enable': {'retrieve_databundle': False, 'download_osm_data': False, 'build_cutout': False, 'build_natura_raster': False}, 'crs': {'geo_crs': 'EPSG:4326', 'distance_crs': 'EPSG:3857', 'area_crs': 'ESRI:54009'}, 'retrieve_databundle': {'show_progress': False}, 'augmented_line_connection': {'add_to_snakefile': False, 'connectivity_upgrade': 2, 'new_line_type': ['HVAC'], 'min_expansion': 1, 'min_DC_length': 600}, 'cluster_options': {'alternative_clustering': False, 'distribute_cluster': ['load'], 'out_logging': True, 'aggregation_strategies': {'generators': {'p_nom': 'sum', 'p_nom_max': 'sum', 'p_nom_min': 'sum', 'p_min_pu': 'mean', 'marginal_cost': 'mean', 'committable

In [2]:
#I create the bus 

n.madd("Bus", ["onebus"], x=0.1, y=0.2, carrier="AC", v_nom=20)



Index(['onebus'], dtype='object')

In [3]:
costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

In [4]:
tech_costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

idx = pd.IndexSlice

def calculate_annuity(n, r):
    """
    Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20, 0.05) * 20 = 1.6
    """
    if isinstance(r, pd.Series):
        return pd.Series(1 / n, index=r.index).where(
            r == 0, r / (1.0 - 1.0 / (1.0 + r) ** n)
        )
    elif r > 0:
        return r / (1.0 - 1.0 / (1.0 + r) ** n)
    else:
        return 1 / n

In [5]:
tech_costs='C://Users//denis//OneDrive//Desktop//Mini grids//pypsa-distribution//costs.csv'

In [6]:
def load_costs(tech_costs, config, elec_config, Nyears=1):
    """
    set all asset costs and other parameters
    """
    costs = pd.read_csv(tech_costs, index_col=list(range(3))).sort_index()

    # correct units to MW and EUR
    costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
    costs.loc[costs.unit.str.contains("USD"), "value"] *= config["USD2013_to_EUR2013"]

    costs = (
        costs.loc[idx[:, config["year"], :], "value"]
        .unstack(level=2)
        .groupby("technology")
        .sum(min_count=1)
    )

    costs = costs.fillna(
        {
            "CO2 intensity": 0,
            "FOM": 0,
            "VOM": 0,
            "discount rate": config["discountrate"],
            "efficiency": 1,
            "fuel": 0,
            "investment": 0,
            "lifetime": 25,
        }
    )

    costs["capital_cost"] = (
        (
            calculate_annuity(costs["lifetime"], costs["discount rate"])
            + costs["FOM"] / 100.0
        )
        * costs["investment"]
        * Nyears
    )

    costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
    costs.at["CCGT", "fuel"] = costs.at["gas", "fuel"]

    costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

    costs = costs.rename(columns={"CO2 intensity": "co2_emissions"})

    costs.at["OCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]
    costs.at["CCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]

    costs.at["solar", "capital_cost"] = 0.5 * (
        costs.at["solar-rooftop", "capital_cost"]
        + costs.at["solar-utility", "capital_cost"]
    )

    def costs_for_storage(store, link1, link2=None, max_hours=1.0):
        capital_cost = link1["capital_cost"] + max_hours * store["capital_cost"]
        if link2 is not None:
            capital_cost += link2["capital_cost"]
        return pd.Series(
            dict(capital_cost=capital_cost, marginal_cost=0.0, co2_emissions=0.0)
        )

    max_hours = elec_config["max_hours"]
    costs.loc["battery"] = costs_for_storage(
        costs.loc["battery storage"],
        costs.loc["battery inverter"],
        max_hours=max_hours["battery"],
    )
    costs.loc["H2"] = costs_for_storage(
        costs.loc["hydrogen storage"],
        costs.loc["fuel cell"],
        costs.loc["electrolysis"],
        max_hours=max_hours["H2"],
    )

    for attr in ("marginal_cost", "capital_cost"):
        overwrites = config.get(attr)
        if overwrites is not None:
            overwrites = pd.Series(overwrites)
            costs.loc[overwrites.index, attr] = overwrites

    return costs

In [7]:
costs = load_costs(
    tech_costs,
    config["costs"],
    config["electricity"],
    Nyears,
    )

In [8]:
def attach_wind_and_solar(n, costs, tech_modelling, extendable_carriers):
    
    for tech in tech_modelling:
        
        # with xr.open_dataset(technologies_path[tech]) as ds:
        with xr.open_dataset(f"renewable_profiles/profile_{tech}.nc") as ds:
            
            if ds.indexes["bus"].empty:
                continue   

            suptech = tech.split("-", 2)[0]
            
            n.madd(
            "Generator",
            ds.indexes["bus"],
            " " + tech,
            bus="onebus",
            carrier=tech,
            p_nom_extendable=tech in extendable_carriers["Generator"],
            p_nom_max=ds["p_nom_max"].to_pandas(), #guardare il config 
            weight=ds["weight"].to_pandas(),
            marginal_cost=costs.at[suptech, "marginal_cost"],
            capital_cost=costs.at[tech, "capital_cost"],
            efficiency=costs.at[suptech, "efficiency"],
            p_set=ds["profile"].transpose("time", "bus").to_pandas().reindex(n.snapshots),
            p_max_pu=ds["profile"].transpose("time", "bus").to_pandas().reindex(n.snapshots),
            )


In [9]:
attach_wind_and_solar(
    n,
    costs,
    config["tech_modelling"]["general_vre"],
    config["electricity"]["extendable_carriers"],
    )

In [10]:
n.madd("StorageUnit",["StorUni"], bus="onebus",  p_nom=5000, marginal_cost=10, inflow=50, p_nom_extendable=True, capital_cost=50, 
       efficiency_dispatch=0.5, cyclic_state_of_charge=True, state_of_charge_initial=1000)

Index(['StorUni'], dtype='object')

In [11]:

#I create the load file
n_load=35 #number of loads
import numpy as np
p_set=np.random.rand(len(n.snapshots),(n_load))
array=p_set*100

In [12]:
import pandas as pd
df=pd.DataFrame(array)

filepath = 'my_file.xlsx'

df.to_excel(filepath, index=False)



In [13]:
load_df=pd.read_excel(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\my_file.xlsx')


In [14]:

load_df=load_df.set_index([n.snapshots])
load_df.index.names=['time']
print(load_df)

                            0          1          2          3          4   \
time                                                                         
2013-01-01 00:00:00  79.813769  74.712218  92.666422  98.330181  61.708476   
2013-01-01 01:00:00  85.188637  17.944634  40.105956  66.820024  67.409571   
2013-01-01 02:00:00   2.898011  88.125592  19.284309  48.356550  79.129995   
2013-01-01 03:00:00   9.231015  49.567249  52.027549  26.966664  41.824124   
2013-01-01 04:00:00  25.386436  88.270797  71.087385  49.660325  37.754316   
...                        ...        ...        ...        ...        ...   
2013-12-31 19:00:00  60.730525  82.191801  59.257181  22.172051  76.604157   
2013-12-31 20:00:00  41.095568  79.681751   3.964934  67.780946  57.995353   
2013-12-31 21:00:00  40.032405  95.499080  53.866610  37.708043  74.881593   
2013-12-31 22:00:00  46.807798  43.213944   4.840567  90.957187  10.580066   
2013-12-31 23:00:00  35.819429  25.218496  60.016971  91.613577 

In [15]:
index=pd.Index( list(range(n_load)))
type(index)

pandas.core.indexes.numeric.Int64Index

In [16]:
def attach_load(n, load_paths, index, load_df):
    
    n.madd("Load", index ,bus=["onebus"], carrier="AC", p_set=load_df)




In [17]:
load_paths=r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\my_file.xlsx'

In [18]:
attach_load(n, load_paths, index, load_df)

In [19]:
n.loads

,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
0,onebus,AC,,0.0,0.0,-1.0
1,onebus,AC,,0.0,0.0,-1.0
2,onebus,AC,,0.0,0.0,-1.0
3,onebus,AC,,0.0,0.0,-1.0
4,onebus,AC,,0.0,0.0,-1.0
5,onebus,AC,,0.0,0.0,-1.0
6,onebus,AC,,0.0,0.0,-1.0
7,onebus,AC,,0.0,0.0,-1.0
8,onebus,AC,,0.0,0.0,-1.0


In [20]:
len(f"renewable_profiles/solar")

24

In [21]:
print(n)

PyPSA Network
Components:
 - Bus: 1
 - Generator: 105
 - Load: 35
 - StorageUnit: 1
Snapshots: 8760


In [22]:
# Optimization
from pypsa.linopf import ilopf

solver_name="gurobi"

n.lopf(n.snapshots, solver_name=solver_name, pyomo=False)

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 6.65s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-07
Read LP format model from file C:\Users\denis\AppData\Local\Temp\pypsa-problem-sessc8iv.lp
Reading time = 4.08 seconds
obj: 1909680 rows, 954947 columns, 2909383 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1909680 rows, 954947 columns and 2909383 nonzeros
Model fingerprint: 0x3a3d7cd7
Coefficient statistics:
  Matrix range     [1e-02, 2e+00]
  Objective range  [1e-02, 1e+05]
  Bounds range     [4e+00, 3e+05]
  RHS range        [5e+01, 2e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 1880829 rows and 926134 columns
Presolve time: 1.48s
Presolved: 28851 rows, 28813 columns, 132585 nonzeros

Ordering time: 0.05s

Barrier statistics:
 Dense cols : 53
 AA' NZ     : 2.382e+05
 Factor NZ  : 6.273e+05 (roughly 30 MB of memory)
 Factor

INFO:pypsa.linopf:Optimization successful. Objective value: 6.04e+10


('ok', 'optimal')